In [ ]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import pint.derived_quantities as dq
import table_utils as tu

pint.logging.setup(level="ERROR")

**J1221-0633 eclipsing calculations**

For J1221-0633, we see additional delays around superior conjuntion at 820 MHz, probably due to additional electron content around the companion that the signal must travel through when the pulsar eclipses (no signal at 350 MHz). First I want to figure out the DM bump corresponding to this delay (roughly 200 microseconds).

In [ ]:
def SC_dDM(dt,freq):
    """ Calculate additional column density at superior conjunction (given additional timing delay)
    
    Parameters
    ==========
    dt: quantity, additional dispersive timing delay due to plasma around companion
    freq: quantity, observing frequency of measured delay
    
    Returns
    =======
    dDM_sup: quantity, additional column density at superior conjunction (cm^-2)
    """
    Dconst = 4.148808e3*(1e6*u.Hz)**2*u.cm**3*u.s*u.pc**-1 # HBOPA, eqn. 4.6
    dDM_sup = (dt*freq**2/Dconst).to(u.pc*u.cm**-3) # extra DM corresponding to delay
    return dDM_sup.to(u.cm**-2)

To compare to a similar calculation in Freire (2005; ASPCS, p. 407), I convert DM (column density) to 1/cm^2. The result is comparable to the column density of J2051-0827 (Stappers et al. 2001).

In [ ]:
dDM_1221 = SC_dDM(200.0*1e-6*u.s,820.0*(1e6*u.Hz))
dDM_1221

How big might the companion be based on the duration of the eclipse? Assume inclination angle of 90 deg for now, and consider an eclipse that lasts some fraction of an orbit (based on gap between ingress-egress).

In [ ]:
def Rplasma(a, frac_eclipse):
    """ Estimate companion (plasma?) radius based on extent of eclipse
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    frac_eclipse: float, fraction of the orbit where pulsar signal is obscured
    
    Returns
    =======
    Rplasma: quantity, companion's plasma(?) radius (Solar Radii)
    """
    print(f"Nominal pulsar-companion separation: {a.value:.4f} Rsun")
    ing2sup = frac_eclipse*180.0*u.deg # angle from ingress to superior conjunction
    Rplasma = a*np.sin(ing2sup)
    print(f"Companion's estimated plasma radius: {Rplasma.value:.4f} Rsun")
    return Rplasma

eclipsing_psrs = ["J1317-0157","J1221-0633"]
frac_eclipses = [0.15,0.16]
rpDict = {}

for ep,fe in zip(eclipsing_psrs,frac_eclipses):
    par_path = f"data/{ep}_swiggum+22.par"
    tim_path = f"data/{ep}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    a = mo['A1'].quantity.to(u.R_sun) # assume 90 deg inclination
    print(f"{ep} is obscured for {fe:.0%} of an orbit...")
    rp = Rplasma(a,fe)
    rpDict[mo['PSR'].value]=rp
    if ep != eclipsing_psrs[-1]: print()

For J1221, I can use the estimated companion's plasma radius to get some idea of the electron density near the companion (using the additional column density around superior conjunction). This is proabbly ~order of magnitude.

In [ ]:
(dDM_1221/rpDict['J1221-0633']).to(u.cm**-3)

Next, I want to use the equation from Eggleton et al. (1983) to estimate the size of the companion's Roche Lobe, and thus, the path length the pulsar might be encountering around superior conjunction. Do this for a few possible inclination angles and assume pulsar mass is 1.4 solar masses.

In [ ]:
def RL_Eggleton(a,mc,mp=1.4*u.solMass):
    """ Calculate Roche Lobe size in solar radii (see Eggleton 1983)
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    mc: quantity, companion mass (solar masses)
    mp: pulsar mass (default: 1.4 Msun), optional
    
    Returns
    =======
    R_L: quantity, Roche Lobe size (solar radii)
    """
    q = mc/mp
    R_L = 0.49*a*q**(2.0/3)/(0.6*q**(2.0/3)+np.log(1.0+q**(1.0/3)))
    return R_L.to(u.R_sun)

eclipsing_psrs = ["J1317-0157","J1221-0633"]
incs = [90.0,80.0,70.0,60.0] # trial inclination angles, degrees

for ep in eclipsing_psrs:
    par_path = f"data/{ep}_swiggum+22.par"
    tim_path = f"data/{ep}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    print(f"Calculations for {ep}...")
    
    for inc in incs:
        a = (mo['A1'].quantity * np.sin(inc*u.deg)).to(u.R_sun)
        mp = 1.4*u.M_sun
        mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc*u.deg, mp=mp)
        R_L = RL_Eggleton(a,mc,mp=mp)
        RlRp_ratio = R_L/rpDict[ep]
        
        mc_str = f"{mc.value:.4f}"
        q_str = f"{mc/mp:.4f}"
        a_str = f"{a.value:.4f}"
        rl_str = f"{R_L.value:.5f}"
        rlrp_str = f"{RlRp_ratio:.2f}"
        
        print(f"  i = {inc} deg; mc = {mc_str} Msun; q = {q_str}; a = {a_str} Rsun; RL = {rl_str} Rsun; RL/Rp = {rlrp_str}")
        
    if ep != eclipsing_psrs[-1]: print()

Therefore (as in Sec. 2.1 of Freire 2005), the Roche Lobe (R_L above) is much smaller than the size of the plasma cloud (from rpDict), and matter responsible for increased dispersive delays is not bound to the companion (i.e. companion is losing mass). This is true for both J1221 and J1317.

**Optical Constraints Calculations**

I follow a procedure similar to that described in Section 7 of Lynch et al. (2018). For non-WD companions, I include a couple functions to convert magnitude limits to limits on effective temperature, but I'm not sure this is quite good enough since Covey 2007 (where I establish spectral types) doesn't include bolometric corrections.

In [ ]:
def MagLim_to_Lum(Mlim):
    """ Convert magnitude limit to luminosity
    
    Parameter
    =========
    Mlim (float): limiting magnitude(s)
    
    Returns
    =======
    Llim (quantity): luminosity limit(s), solar luminosity
    """
    Msun = 4.74
    Llim=u.solLum*10**((Msun-Mlim)/2.5)
    return Llim.decompose()

def TeffLim(Rlim,Llim):
    """ Limit effective temp based on radius, luminosity
    
    Parameters
    ==========
    Rlim (quantity): estimate of stellar radius, solar radii
    Llim (quantity): estimate of stellar luminosity, solar luminosity
    
    Returns
    =======
    Tlim (quantity): upper limit on Teff, Kelvin
    """
    from astropy.units.cds import c
    sbc = 5.6704e-8*u.W*u.meter**-2*u.K**-4
    Tlim=(Llim/(4*np.pi*Rlim**2*sbc))**0.25
    return Tlim.decompose()

In [ ]:
# extinction from http://argonaut.skymaps.info (see Green et al. 2015)
# these take distance (largest NE vs. YMW) into account
EBmV_dict = {
    "J0742+4110":0.05,
    "J1045-0436":0.03,
    "J1221-0633":0.03,
    "J1317-0157":0.01,
    "J2022+2534":0.23,
    "J2039-3616":0.0, # justification to ignore reddening?
}

# Read Covey 2007 table (assume spectral classification V - main sequence)
with open('data/covey07.txt') as covey:
   covey_info = covey.readlines()

SpectType_griz_Dict = {}
for ci in covey_info:
    ci_list = ci.strip().split(' ')
    spect_type = ci_list[0]
    MJ = float(ci_list[-1])
    z = float(ci_list[5])+MJ
    i = float(ci_list[4])+z
    r = float(ci_list[3])+i
    g = float(ci_list[2])+r
    #print(f"{spect_type:4s}: {g:.2f}, {r:.2f}, {i:.2f}, {z:.2f}")
    
    SpectType_griz_Dict[spect_type] = [g,r,i,z]

# reddening in PS1 bands from Schlafly & Finkbeiner 2011, Table 6
# grizy
ps1_redden=np.array([3.172,2.271,1.682,1.322]) # ,1.087]) y not constraining (and not common to skymapper)

# assume no detection.  Typical PS1 griz stack limits are:
ps1_lims = np.array([23.3,23.2,23.1,22.3]) # ,21.4])
# Skymapper DR2 stack lims (griz); Onken et al 2019
sm_lims = np.array([22.0,22.0,21.0,20.0])

for psr in EBmV_dict.keys():
    par_path = f"data/{psr}_swiggum+22.par"
    mo = model.get_model(par_path)
    
    mp = 1.4*u.M_sun
    mcmed = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=60.0*u.deg, mp=mp)
    mcmin = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=90.0*u.deg, mp=mp)
    
    gcoord = mo.coords_as_GAL()
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,mo['DM'])
    
    if psr == "J1317-0157":
        dmdist = dmdist_ne # ymw distance estimate maxed out
    else:
        dmdist = max([dmdist_ne,dmdist_ymw])
    
    distance_mod = 5*np.log10((dmdist*u.kpc/(10.0*u.pc)).decompose())
    if psr == "J2039-3616":
        print(f"{psr}: {mcmin.value:.3f}-{mcmed.value:.3f} Msun")
        print(sm_lims-distance_mod)
    else:
        print(f"{psr}: {mcmin.value:.3f}-{mcmed.value:.3f} Msun")
        griz_lims = ps1_lims-EBmV_dict[psr]*ps1_redden-distance_mod # de-reddened limits
        print(griz_lims)
        if psr != "J1045-0436":
            a = (mo['A1'].quantity * np.sin(60.0*u.deg)).to(u.R_sun) # assume i = 60 deg
            mp = 1.4*u.M_sun
            mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=60.0*u.deg, mp=mp)
            R_L = RL_Eggleton(a,mc,mp=mp)
            print(f"Radius (RL) estimate: {R_L}") # why is this important?
            
            # check griz_lims against SpectType_griz_Dict
            # g=0, r=1, i=2, z=3
            # g is always the most limiting, so range(1) rather than range(3)
            for band_i in range(1): 
                for stk in SpectType_griz_Dict.keys():
                    if griz_lims[band_i] < SpectType_griz_Dict[stk][band_i]:
                        print(f"{['g','r','i','z'][band_i]} spectral type limit: {stk} ({griz_lims[band_i]}<{SpectType_griz_Dict[stk][band_i]})")
                        Tlim = TeffLim(R_L,MagLim_to_Lum(SpectType_griz_Dict[stk][band_i]))
                        print(f"Teff limit?: {Tlim.decompose()}")
                        break
        
    print(dmdist,distance_mod)

I think the issue with temperature constraints reported here for M-stars is that they're based on band-specific magnitude to luminosity limits. What I want to do instead is, once I've determined the correct band/spectral type to do the limiting, use the bolometric correction (??) to convert the band-limiting (g-band across the board it looks like) magnitude to a bolometric magnitude, convert to luminosity, and THEN set a Teff limit with R_L. For now, I'll just grab estimates for Teff from the web associated with M6, M5, M3, M1. These limits and spectral types assume main sequence stars (V) so I think all of this is already really uncertain.

Teff for M6 = 3100 K, M5 = 3200 K, M3 = 3500 K, M1 = 3700 K.
Grabbed these from https://sites.uni.edu/morgans/astro/course/Notes/section2/spectraltemps.html

In [ ]:
import matplotlib.pyplot as plt
from astropy.units import cds

# Didn't end up using Istrate 2016 radius correction
r_i16,teff_i16,m_i16,xxx,age_i16=np.loadtxt("data/radius_teff_mass.txt",dtype='float',unpack=True)
#set(m_i16) # which WD masses are modeled here?
inds_0p2=np.where(m_i16==0.1961)

# read in example Bergeron 2011 m=0.2 Msun table (He; https://www.astro.umontreal.ca/~bergeron/CoolingModels/)
bergdata=np.loadtxt('data/berg0.2.txt')
bteff = bergdata[:,0]
blogg = bergdata[:,1]
ber_radii_m=(cds.G*0.2*u.solMass/(10**blogg*u.meter/u.s**2)).decompose().value**0.5

m_kg = (m*u.solMass).to(u.kg)
r_m = (r_i16[inds]*u.solRad).to(u.meter)
logg = np.log10((cds.G*m_kg/r_m**2).decompose().value)

plt.plot(logg,teff_i16[inds]*1e3,label='Istrate 2016')
plt.plot(blogg,bteff,label='Bergeron 2011')
plt.legend()

#plt.plot(np.log10(r_m.value),teff_i16[inds]*1e3,label='Istrate 2016')
#plt.plot(np.log10(ber_radii_m),bteff,label='Bergeron 2011')
#plt.legend()

In [ ]:
# Implement Mamajek values instead (include BC, but how to get griz from colors provided?)
with open('data/mamajek.txt') as mamajek:
    mj_data = mamajek.readlines()

In [ ]:
for mj in mj_data:
    if mj.startswith("#"):
        continue
    print(mj)